In [ ]:
from google.colab import files
uploaded = files.upload()
import pandas as pd
df = pd.read_csv('data_new.csv')
data = df.copy()

Saving data_new.csv to data_new.csv


In [ ]:
df['Datetime'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
df

,Year,Month,Day,Temperature,Humidity,Datetime
0,2021,12,1,14.83,67.25,2021-12-01
1,2021,12,2,13.58,70.50,2021-12-02
2,2021,12,3,14.93,66.88,2021-12-03
3,2021,12,4,15.58,67.00,2021-12-04
4,2021,12,5,15.62,68.56,2021-12-05
...,...,...,...,...,...,...
742,2023,12,13,22.38,90.88,2023-12-13
743,2023,12,14,23.23,89.38,2023-12-14
744,2023,12,15,22.38,90.88,2023-12-15
745,2023,12,16,14.01,64.00,2023-12-16


In [13]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA

df['Datetime'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
df.set_index('Datetime', inplace=True)

# Predict temperature
# Predict temperature
last_date = df.index[-1]
future_days = 15 # change this to adjust the number of predicted days
start_date = last_date + pd.Timedelta(days=1)
end_date = start_date + pd.Timedelta(days=future_days - 1)
model = ARIMA(df['Temperature'].dropna(), order=(1, 0, 0))
model_fit = model.fit()
predicted_temperature = model_fit.predict(start=last_date, end=start_date + pd.Timedelta(days=future_days - 1), dynamic=True)

# Predict humidity
model = ARIMA(df['Humidity'].dropna(), order=(1, 0, 0))
model_fit = model.fit()
predicted_humidity = model_fit.predict(start=last_date, end=start_date + pd.Timedelta(days=future_days - 1), dynamic=True)

# Get the first 5 predicted values
first_5_temperature = predicted_temperature.head(5)
first_5_humidity = predicted_humidity.head(5)
print(first_5_temperature.values)
print(first_5_humidity.values)
predict = list(first_5_temperature.values) + list(first_5_humidity.values)
predict = str(predict)[1:-1]
print(predict)




/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dat

[14.44905713 14.86627819 15.26274918 15.63950208 15.99751756]
[69.38535046 73.29255487 76.12732791 78.18402557 79.67621039]
14.449057126804535, 14.866278188398137, 15.262749179551843, 15.63950208417753, 15.997517561503745, 69.38535046131612, 73.2925548698936, 76.1273279083555, 78.18402557227839, 79.67621038686268


In [14]:
!pip install paho-mqtt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for paho-mqtt: filename=paho_mqtt-1.6.1-py3-none-any.whl size=62120 sha256=3818e91ebed22793e77d27a177702a80cf98ea847937256c64ee24dd54bb18fe
  Stored in directory: /root/.cache/pip/wheels/8b/bb/0c/79444d1dee20324d442856979b5b519b48828b0bd3d05df84a
Successfully built paho-mqtt


In [ ]:
humi = 0
temp = 0
import paho.mqtt.client as mqtt
import time
client = mqtt.Client()

def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Kết nối thành công")
        client.subscribe("ESP32WI/DHT11/Humidity01")
        client.subscribe("ESP32WI/DHT11/Temperature01")
    else:
        print("Kết nối thất bại, mã lỗi:", rc)

client.on_connect = on_connect

broker_address = "broker.mqttdashboard.com"
username = "lamlam"
password = "Aa123456"
client.username_pw_set(username, password)
client.connect(broker_address, 1883)

# client.loop_forever()

while True:
      topic = "predict5"
      client.publish(topic, predict)
      time.sleep(10)